In [10]:
##### 10-1 & 10-2 CNN 연습문제 #####

import torch
import torch.nn as nn

# 예제 1
# input image size : 227 x 227
# filter size : 11 x 11
# stride : 4
# padding : 0
# output image size = ?


# nn.Conv2d(input_channel,output_channel,stride,padding)
conv = nn.Conv2d(1,1,11,stride=4,padding=0)

# input format= (batchsize, channel, height, width)
inputs=torch.Tensor(1,1,227,227)

out = conv(inputs)
out.shape

torch.Size([1, 1, 55, 55])

In [13]:
conv = nn.Conv2d(1,1,7,stride=2,padding=0)
inputs = torch.Tensor(1,1,64,64)

out= conv(inputs)
out.shape

torch.Size([1, 1, 29, 29])

In [15]:
conv = nn.Conv2d(1,1,5,stride=1,padding=2)
inputs = torch.Tensor(1,1,32,32)

out=conv(inputs)
out.shape

torch.Size([1, 1, 32, 32])

In [16]:
conv = nn.Conv2d(1,1,5,stride=1,padding=0)
inputs = torch.Tensor(1,1,32,64)

out=conv(inputs)
out.shape

torch.Size([1, 1, 28, 60])

In [17]:
conv = nn.Conv2d(1,1,3,stride=1,padding=1)
inputs = torch.Tensor(1,1,64,32)

out = conv(inputs)
out.shape

torch.Size([1, 1, 64, 32])

In [62]:
inputs = torch.Tensor(1,1,28,28)
conv1 = nn.Conv2d(1,5,5)
pool = nn.MaxPool2d(2,2)
out = conv1(inputs)
out2 = pool(out)
out.size()
out2.size()

torch.Size([1, 5, 12, 12])

In [32]:
inputs = torch.Tensor(1,1,28,28)
conv1 = nn.Conv2d(1,32,3,stride=1,padding=1)
pool =nn.MaxPool2d(2)
conv2 = nn.Conv2d(32,64,3,stride=1,padding=1)
out = conv1(inputs)
out= pool(out)
out = conv2(out)
out = pool(out)
# torch.Size([1,64,7,7])
out = out.view(out.size(0),-1) # out.size(0) = 1
fc = nn.Linear(3136,10)
out = fc(out)
out.shape

torch.Size([1, 10])

In [49]:
##### 10-2 CNN 예제 #####
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [50]:
# visdom #
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [51]:
# visdom #
# define loss_tracker #
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [63]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

cpu


In [53]:
#파라미터
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [54]:
# MNIST dataset

mnist_train = dsets.MNIST(root ='MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)


In [55]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [56]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        self.fc = torch.nn.Linear(7*7*64,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out

In [57]:
model = CNN().to(device)

In [66]:
# loss function & optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [59]:
# visdom #
# make plot #
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))
# loss_plt 선언 = vis.line() -> line plot 생성
# (0을 넣음, 이름 = loss_tracker, line 이름 = loss, 값을 보여달란 의미)

In [60]:
# train my model
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1,avg_cost))
    # loss_trakcer: loss_plt 업데이트 함수
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
    # (plt이름, tensor형태로 avg_cost업데이트, 업데이트할 인덱스)
print('Learning Finished!')

[Epoch:    1] cost = 0.225607827
[Epoch:    2] cost = 0.0629417673
[Epoch:    3] cost = 0.0462888777
[Epoch:    4] cost = 0.0374611467
[Epoch:    5] cost = 0.0313362107
[Epoch:    6] cost = 0.0260468703
[Epoch:    7] cost = 0.0219094232
[Epoch:    8] cost = 0.0185330287
[Epoch:    9] cost = 0.0164066143
[Epoch:   10] cost = 0.0133768078
[Epoch:   11] cost = 0.0100030983
[Epoch:   12] cost = 0.01024774
[Epoch:   13] cost = 0.00819545612
[Epoch:   14] cost = 0.00696233194
[Epoch:   15] cost = 0.00620753365
Learning Finished!


In [61]:
with torch.no_grad(): ## history 트래킹하지 않겠다
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:',accuracy.item())

/Users/ohgyulim/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/ohgyulim/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9887999892234802
